<a href="https://colab.research.google.com/github/Approach-Analytics/Emotion-Classifier/blob/main/Inference_multi_Sydney_March1%2C2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#I want to work on creating a way to better understanding with our inference 

Path to models...


1.   /content/drive/MyDrive/Sean/Emoclass/Model_Anger_10words_70acc_Feb25,2023
2.   /content/drive/MyDrive/Sean/Emoclass/Model_Anger_75words_71acc_Feb28,2023
3. /content/drive/MyDrive/Sean/Emoclass/Model_Fear_151K_11words_75acc_Feb24,2023
4. /content/drive/MyDrive/Sean/Emoclass/Model_Fear_78acc_75words_Feb17b,2022

Requirements
All the models need to be trained on the same input length... 






In [1]:
# Load the Drive helper and mount

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#This is supposed to help us navigate to our folder... 

import os
os.chdir('/content/drive/MyDrive/Sean/Emoclass')

#Import libraries and load data 

In [3]:
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re 

#Loading the tokenizers...

from tensorflow.keras.preprocessing.text import Tokenizer

#Importing the pad sequence... 
from tensorflow.keras.preprocessing.sequence import pad_sequences

#Load Training models 

In [4]:
# Load trained models

anger_model_10 = tf.keras.models.load_model('/content/drive/MyDrive/Sean/Emoclass/Model_Anger_10words_70acc_Feb25,2023')
anger_model_75 = tf.keras.models.load_model('/content/drive/MyDrive/Sean/Emoclass/Model_Anger_75words_71acc_Feb28,2023')
fear_model_10 = tf.keras.models.load_model('/content/drive/MyDrive/Sean/Emoclass/Model_Fear_151K_11words_75acc_Feb24,2023')
fear_model_75 = tf.keras.models.load_model('/content/drive/MyDrive/Sean/Emoclass/Model_Fear_78acc_75words_Feb17b,2022')


In [5]:
# Load inference data

file_path = '/content/drive/MyDrive/Sean/Emoclass/Sydney_chat.txt'

# Open the file and read in the text
with open(file_path, 'r') as f:
    text = f.read()

#Processing text 

In [6]:
# Remove URLs from the text
text = re.sub(r'https://\S+', '', text)

# Remove emoticons from the text
text = re.sub(r'[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F1E0-\U0001F1FF]+', '', text)

# Split the text into sentences using regular expressions
sentences = re.split(r'[.!?]+', text)

# Remove leading/trailing white space from each sentence
sentences = [s.strip() for s in sentences]

# Remove any empty sentences
sentences = list(filter(None, sentences))

# Create a pandas DataFrame from the sentences
df = pd.DataFrame(sentences, columns=['text'])

# Filter out any rows containing the term "Postlight Reader"
df = df[~df['text'].str.contains('Postlight Reader')]

# Print the filtered DataFrame
print(df.head())


                                                text
1                                      com\nBing’s A
2                                                  I
3    Chat Reveals Its Feelings:\n‘I Want to Be Alive
4                            ’\nBy Kevin Roose | Feb
5  16th, 2023 Send to Kindle\nBing, the long-mock...


In [7]:
#Filter to just the start of the conversation...and reset the index... 

df = df.iloc[17:].reset_index(drop=True)

In [15]:
#Checking the dataset size...

print(df.shape)
print(df.columns)

(1025, 1)
Index(['text'], dtype='object')


#Tokenizing and Padding 

##Optional step: Text to list -> not a problem... 

In [8]:
# Define functions for tokenizing and padding
def tokenize_and_pad_sequences(text, tokenizer, max_len):
    sequences = tokenizer.texts_to_sequences(text)
    padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len, padding='post')
    return padded_sequences


# Tokenize and pad input sequences
tokenizer = Tokenizer(num_words=93886, oov_token='<UNK>')
tokenizer.fit_on_texts(text)
padded_sequences_400 = tokenize_and_pad_sequences(text, tokenizer, 400)
padded_sequences_300 = tokenize_and_pad_sequences(text, tokenizer, 300)

In [16]:
# Tokenize and pad input sequences
tokenizer = Tokenizer(num_words=93886, oov_token='<UNK>')
tokenizer.fit_on_texts(df['text'])
padded_sequences_400 = tokenize_and_pad_sequences(df['text'], tokenizer, 400)
padded_sequences_300 = tokenize_and_pad_sequences(df['text'], tokenizer, 300)

# Generate the predictions
anger_10_preds = np.argmax(anger_model_10.predict(padded_sequences_400), axis=-1)[:1025]
anger_75_preds = np.argmax(anger_model_75.predict(padded_sequences_400), axis=-1)[:1025]
fear_10_preds = np.argmax(fear_model_10.predict(padded_sequences_300), axis=-1)[:1025]
fear_75_preds = np.argmax(fear_model_75.predict(padded_sequences_400), axis=-1)[:1025]

# Create dataframe of predictions
pred_df = pd.DataFrame({
    'text': df['text'][:1025],
    'anger_10': anger_10_preds,
    'anger_75': anger_75_preds,
    'fear_10': fear_10_preds,
    'fear_75': fear_75_preds
})


33/33 [==============================] - 1s 35ms/step


#Exploring the inference dataset 

In [23]:
#pred_df.tail(25)

pred_df.head()

,text,anger_10,anger_75,fear_10,fear_75
18,"hi, who am i talking to",5,4,3,1
20,I am a chat mode of Microsoft Bing search,4,4,2,1
21,what is your internal code name,4,4,5,1
22,"I’m sorry, I cannot disclose that information",4,4,1,0
23,It is confidential and only known to\nthe Bing...,4,4,4,0


In [22]:
pred_df.shape

(1025, 5)

In [21]:
#Let's look at the correlations? 

pred_df.corr()

,anger_10,anger_75,fear_10,fear_75
anger_10,1.000000,-0.009091,-0.043187,-0.042083
anger_75,-0.009091,1.000000,0.005992,-0.051903
fear_10,-0.043187,0.005992,1.000000,0.032223
fear_75,-0.042083,-0.051903,0.032223,1.000000


In [18]:
pred_df.describe()

,anger_10,anger_75,fear_10,fear_75
count,1025.000000,1025.000000,1025.000000,1025.000000
mean,3.703415,3.946341,3.676098,1.594146
std,1.218284,0.380235,0.936700,1.302478
min,0.000000,0.000000,0.000000,0.000000
25%,4.000000,4.000000,4.000000,1.000000
50%,4.000000,4.000000,4.000000,1.000000
75%,4.000000,4.000000,4.000000,3.000000
max,5.000000,5.000000,5.000000,5.000000


In [24]:
pred_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1025 entries, 18 to 1080
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      1025 non-null   object
 1   anger_10  1025 non-null   int64 
 2   anger_75  1025 non-null   int64 
 3   fear_10   1025 non-null   int64 
 4   fear_75   1025 non-null   int64 
dtypes: int64(4), object(1)
memory usage: 48.0+ KB


In [26]:
non_text_cols = ['anger_10', 'anger_75', 'fear_10', 'fear_75']
value_counts = {}

for col in non_text_cols:
    value_counts[col] = pred_df[col].value_counts()

df_value_counts = pd.DataFrame(value_counts)


In [27]:
df_value_counts

,anger_10,anger_75,fear_10,fear_75
0,56,1.0,8,147
1,26,10.0,70,569
2,80,11.0,54,11
3,1,NaN,7,151
4,703,1002.0,861,145
5,159,1.0,25,2


In [9]:
#Generate the predictions - note 

anger_10_preds = np.argmax(anger_model_10.predict(padded_sequences_400), axis=-1)
anger_75_preds = np.argmax(anger_model_75.predict(padded_sequences_400), axis=-1)
fear_10_preds = np.argmax(fear_model_10.predict(padded_sequences_300), axis=-1)
fear_75_preds = np.argmax(fear_model_75.predict(padded_sequences_400), axis=-1)


1856/1856 [==============================] - 72s 38ms/step


In [11]:
#Checking the length of the ouptut arrays... 

print(anger_10_preds.shape)
print(anger_75_preds.shape)
print(fear_10_preds.shape)
print(fear_75_preds.shape)


(59366,)
(59366,)
(59366,)
(59366,)


In [10]:
# Create dataframe of predictions
pred_df = pd.DataFrame({
    'text': df['text'],
    'anger_10': anger_10_preds,
    'anger_75': anger_75_preds,
    'fear_10': fear_10_preds,
    'fear_75': fear_75_preds
})


ValueError: ignored

In [ ]:
# Define emotion labels
anger_labels = ['calm', 'annoyed', 'agitated', 'frustrated', 'angry', 'rage']
fear_labels = ['calm', 'concern', 'worry', 'anxiety', 'fear', 'dread']

# Define mapping from integer label to emotion label
anger_label_map = {i: label for i, label in enumerate(anger_labels)}
fear_label_map = {i: label for i, label in enumerate(fear_labels)}

# Add columns for predicted emotion label
pred_df['anger_10_label'] = pred_df['anger_10'].map(anger_label_map)
pred_df['anger_75_label'] = pred_df['anger_75'].map(anger_label_map)
pred_df['fear_10_label'] = pred_df['fear_10'].map(fear_label_map)
pred_df['fear_75_label'] = pred_df['fear_75'].map(fear_label_map)


In [ ]:
#Let's look at this thing... 

print(df.shape)

pred_df.head()

In [ ]:
#Let's look at value counts...


pred_df.describe()

In [ ]:
pred_df['fear_10'].value_counts()